In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.baseline_rank.baseline_rank import BaselineRank
from scripts.models.ltr.train import LTRTrain
from scripts.extractive_summary.ltr.ltr_features_targets import LTRFeaturesTargets
from scripts.models.ltr_gbt.train import LTRGBTTrain
from scripts.models.ltr_svm_tf.train import LTRSVMTFTrain
from scripts.models.ltr_nn_tf.train import LTRNNTFTrain
from scripts import conf
%reload_ext autoreload

In [4]:
import pandas as pd

# Evaluación

Se usará el mismo desarrollo que se utilizó cuando se calcularon las métricas de los resúmenes generados por eventos clave y por grafos
conceptuales

# Baseline

## Carga de datos

In [20]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'cosine_tfidf'
drop_teams = True
lemma = True
metric_params = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [21]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to cosine_tfidf


In [22]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [23]:
rank.run()

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv


In [24]:
rank.ltr.target_metric

'cosine_tfidf'

## Evaluación

In [25]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [26]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_avg.pickle


In [27]:
avg_scores_dict

{'rouge-1': {'f': 0.19979229468420182,
  'p': 0.24900335591362643,
  'r': 0.1948519608845315},
 'rouge-2': {'f': 0.043968174546395476,
  'p': 0.05707513269938273,
  'r': 0.04218564407361283},
 'rouge-l': {'f': 0.1885891367106406,
  'p': 0.3205694129713662,
  'r': 0.14408239323536431}}

### Con procesado

In [28]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_processed_avg.pickle


In [29]:
avg_scores_dict

{'rouge-1': {'f': 0.20614710520254323,
  'p': 0.2370105533948301,
  'r': 0.21777546369970727},
 'rouge-2': {'f': 0.06223472563933338,
  'p': 0.0705535173269253,
  'r': 0.06670498029156349},
 'rouge-l': {'f': 0.23652474459866485,
  'p': 0.37782913410707086,
  'r': 0.1873853769149725}}

# Random forest

## Carga de datos

In [5]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'cosine_emb',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
num_features = ['tfidf_sum']
cat_features_dict = {'is_key_event': [0, 1],
                     'n_players_cat': ['no_player', 'one_player', 'more_than_one_player']}

model_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [2, 5, 7],
    'bootstrap': [True],
    
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [20]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to cosine_tfidf


In [6]:
#CV
ltr_train = LTRTrain(cat_features_dict=cat_features_dict,
                     num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to cosine_emb


In [7]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


## Evaluación

In [8]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [9]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                        preprocess_text=False)

Categorizing n_players...
Computing new length...
Dropping {'equalize', 'advantage', 'position', 'url', 'players_importance', 'total_length', 'n_stop', 'sentence_ix', 'sim_previous_5', 'sim_previous_1', 'score', 'json_file', 'event_ix', 'n_players', 'length', 'sim_previous_3'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/3829e2a65c/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/3829e2a65c/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_avg.pickle


In [10]:
avg_scores_dict

{'rouge-1': {'f': 0.20111834004177417,
  'p': 0.207339101790528,
  'r': 0.23648394422457317},
 'rouge-2': {'f': 0.040140113666721225,
  'p': 0.04278073308561328,
  'r': 0.046591987761209076},
 'rouge-l': {'f': 0.19702606871305084,
  'p': 0.3003660860645218,
  'r': 0.1574808798864388}}

### Con procesado

In [11]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/3829e2a65c/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_cosine_emb_1fab3e681b_processed_avg.pickle


In [12]:
avg_scores_dict

{'rouge-1': {'f': 0.18638521528468227,
  'p': 0.18362965774955928,
  'r': 0.23471851073249891},
 'rouge-2': {'f': 0.05473414077583345,
  'p': 0.0532423333542518,
  'r': 0.07019247954765488},
 'rouge-l': {'f': 0.24468234243213569,
  'p': 0.3537913246600791,
  'r': 0.20296960727975985}}

# GBT

## Carga de datos

In [13]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'cosine_emb',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1]}
num_features = ['tfidf_sum']
model_params = {
    'min_samples_leaf': [50],
    'subsample': [0.8],  
    'learning_rate': [0.1],
    'n_estimators': [80],
    'max_depth':[7],
    'min_samples_split': [600],
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [14]:
ltr_train = LTRGBTTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to cosine_emb


In [15]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840'

## Evaluación

In [16]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [17]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Categorizing n_players...
Computing new length...
Dropping {'equalize', 'advantage', 'n_players_cat', 'position', 'url', 'players_importance', 'total_length', 'n_stop', 'sentence_ix', 'sim_previous_5', 'sim_previous_1', 'score', 'json_file', 'event_ix', 'n_players', 'length', 'sim_previous_3'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/51f2ec7f23/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/51f2ec7f23/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_avg.pickle


In [18]:
avg_scores_dict

{'rouge-1': {'f': 0.201921105085424,
  'p': 0.20833118378406265,
  'r': 0.2370872752585827},
 'rouge-2': {'f': 0.04021670191451893,
  'p': 0.04287918453086039,
  'r': 0.04665393558834069},
 'rouge-l': {'f': 0.19735910062489112,
  'p': 0.29948272041345897,
  'r': 0.15808560787032588}}

### Con procesado

In [19]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/51f2ec7f23/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_processed_avg.pickle


In [20]:
avg_scores_dict

{'rouge-1': {'f': 0.1872711475667253,
  'p': 0.1845313293837474,
  'r': 0.23563921668459256},
 'rouge-2': {'f': 0.05489311154134156,
  'p': 0.05340190428825902,
  'r': 0.07033481429203406},
 'rouge-l': {'f': 0.24501137812334733,
  'p': 0.3521239300846463,
  'r': 0.2039202256949495}}

# SVM

## Carga de datos

In [13]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'cosine_emb',
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}

model_params = {}

In [15]:
ltr_train = LTRSVMTFTrain(model_params=model_params,
                          ltr_params=ltr_params,
                         dim_reduction_params=None)

Setting target metric to cosine_emb
Setting mode to tfidf


In [22]:
ltr_train = LTRSVMTFTrain(model_params=model_params, ltr_params=ltr_params,
                         dim_reduction_params=dim_reduction_params)

Setting target metric to cosine_emb
Setting mode to tfidf
Using truncated_svd for dimensionality reduction


In [16]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/c842b51b47'

## Evaluación

In [17]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [18]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/05f6038bdd/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/05f6038bdd/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_emb_2fcdfe0b57
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_emb_2fcdfe0b57.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_emb_2fcdfe0b57_avg.pickle


In [19]:
avg_scores_dict

{'rouge-1': {'f': 0.21808844796097995,
  'p': 0.2383187650971738,
  'r': 0.23676367843958449},
 'rouge-2': {'f': 0.045325758348459026,
  'p': 0.05124110687240114,
  'r': 0.04852834396428495},
 'rouge-l': {'f': 0.19730096542160275,
  'p': 0.31026793650978696,
  'r': 0.155018436150368}}

### Con procesado

In [20]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/05f6038bdd/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_emb_2fcdfe0b57_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_emb_2fcdfe0b57_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_emb_2fcdfe0b57_processed_avg.pickle


In [21]:
avg_scores_dict

{'rouge-1': {'f': 0.2136826655031749,
  'p': 0.21840286979631152,
  'r': 0.2498562215995},
 'rouge-2': {'f': 0.05919983037931914,
  'p': 0.05943802205064949,
  'r': 0.07082136243086626},
 'rouge-l': {'f': 0.26134007428089834,
  'p': 0.38572728628590036,
  'r': 0.21347179094424446}}

# NN

## Carga de datos

In [39]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'cosine_emb',
    'metric_params': {'embedding': 'distilroberta-base-paraphrase-v1',
                     'text_process': 'basic'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}
DENSE_LAYERS = [125, 60, 20]
N_DROPOUT = 3

model_params = {
    'dense_layers': DENSE_LAYERS,
    'dense_activation': ['relu'] * len(DENSE_LAYERS),
    'dropout': [0.3] * N_DROPOUT,
    'optimizer': 'adam',
}

dim_reduction_params = {
        'dim_reduction': 'truncated_svd',
        'dim_reduction_params': {
            'n_components': 250,
            'random_state': 10
    }
}

opt_metric = 'mean_squared_error'
epochs = 25
batch_size = 128
# Not in config
shuffle = True
max_queue_size = 50

In [32]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric)

Adding 1 zeros so that params have the same length
Setting target metric to cosine_emb
Setting mode to tfidf
Optimizing mean_squared_error with adam
Epochs: 25
Batch size: 128


In [40]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric, 
                        dim_reduction_params=dim_reduction_params)

Using truncated_svd for dimensionality reduction
Using truncated_svd for dimensionality reduction
Setting target metric to cosine_emb
Setting mode to tfidf
Optimizing mean_squared_error with adam
Epochs: 25
Batch size: 128


In [41]:
rank_model = RankModel(ltr_train, n=10, is_nn=True)

Model already trained


In [34]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/48e5a967da'

In [47]:
type(rank_model)

scripts.models.rank_with_model.rank_with_model.RankModel

## Evaluación

In [42]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [43]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Reducing x dimension
Shape before: (423800, 6550)
Shape after: (423800, 250)
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/71c414f12e/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/71c414f12e/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_emb_440cde5154
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_emb_440cde5154.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_emb_440cde5154_avg.pickle


In [44]:
avg_scores_dict

{'rouge-1': {'f': 0.21472022154684792,
  'p': 0.22902253005420478,
  'r': 0.24077421944949634},
 'rouge-2': {'f': 0.04486883758937189,
  'p': 0.04952821168020017,
  'r': 0.04956976950590065},
 'rouge-l': {'f': 0.19875702085702737,
  'p': 0.30951975078630783,
  'r': 0.15696083303790878}}

### Con procesado

In [45]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/71c414f12e/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_emb_440cde5154_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_emb_440cde5154_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_emb_440cde5154_processed_avg.pickle


In [46]:
avg_scores_dict

{'rouge-1': {'f': 0.20685960306113357,
  'p': 0.20887305248481783,
  'r': 0.24786469025409558},
 'rouge-2': {'f': 0.05818409127951038,
  'p': 0.05778476340439716,
  'r': 0.0712352797298071},
 'rouge-l': {'f': 0.2600475100385586,
  'p': 0.38176053706267893,
  'r': 0.21354205829464692}}

# Eventos clave y grafos conceptuales

In [75]:
%%time
evaluation.evaluate_all_summaries()

Evaluating the following summaries: ['key_events_summaries_1.csv', 'key_events_summaries_3.csv', 'key_events_summaries_4.csv', 'key_events_summaries_graph_2.csv', 'key_events_summaries_graph_3.csv', 'key_events_summaries_2.csv', 'key_events_summaries_graph_5.csv', 'key_events_summaries_graph_1.csv', 'key_events_summaries_graph_4.csv']
Evaluating key_events_summaries_1.csv
Metrics already exist
Evaluating key_events_summaries_3.csv
Metrics already exist
Evaluating key_events_summaries_4.csv
Metrics already exist
Evaluating key_events_summaries_graph_2.csv
Metrics already exist
Evaluating key_events_summaries_graph_3.csv
Metrics already exist
Evaluating key_events_summaries_2.csv
Metrics already exist
Evaluating key_events_summaries_graph_5.csv
Metrics already exist
Evaluating key_events_summaries_graph_1.csv
Metrics already exist
Evaluating key_events_summaries_graph_4.csv
Metrics already exist
CPU times: user 1.88 s, sys: 122 ms, total: 2 s
Wall time: 3.86 s


# Todos los resúmenes

In [30]:
pd_metrics = evaluation.output_avg_metrics()

In [31]:
pd.pivot_table(pd_metrics, index='experiment', columns=['metric', 'metric_type'], values='value')

metric                                               rouge-1            \
metric_type                                                f         p   
experiment                                                               
baseline_rank_cosine_tfidf                          0.199792  0.249003   
baseline_rank_cosine_tfidf_processed                0.206147  0.237011   
baseline_rank_rouge-1_r                             0.217420  0.244630   
baseline_rank_rouge-1_r_processed                   0.214708  0.221810   
key_events_summaries_1                              0.152305  0.337611   
key_events_summaries_1_processed                    0.195851  0.520812   
key_events_summaries_2                              0.151979  0.338229   
key_events_summaries_2_processed                    0.195232  0.522998   
key_events_summaries_3                              0.152305  0.337611   
key_events_summaries_3_processed                    0.195851  0.520812   
key_events_summaries_4                              0.151979  0.338229   
key_events_summaries_4_processed                    0.195232  0.522998   
key_events_summaries_graph_1                        0.176339  0.155282   
key_events_summaries_graph_1_processed              0.131490  0.107596   
key_events_summaries_graph_2                        0.151728  0.120522   
key_events_summaries_graph_2_processed              0.122147  0.091738   
key_events_summaries_graph_3                        0.176339  0.155282   
key_events_summaries_graph_3_processed              0.131490  0.107596   
key_events_summaries_graph_4                        0.151728  0.120522   
key_events_summaries_graph_4_processed              0.122147  0.091738   
key_events_summaries_graph_5                        0.173270  0.140934   
key_events_summaries_graph_5_processed              0.134658  0.101599   
rank_with_model_ltr_gbt_cosine_tfidf_433610900a     0.205958  0.212940   
rank_with_model_ltr_gbt_cosine_tfidf_433610900a...  0.192426  0.188675   
rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22        0.202762  0.207199   
rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22_pr...  0.186269  0.181213   
rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8      0.212185  0.236666   
rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8_...  0.209298  0.219853   
rank_with_model_ltr_nn_rouge-1_r_4083eddade         0.214111  0.224458   
rank_with_model_ltr_nn_rouge-1_r_4083eddade_pro...  0.203637  0.199008   
rank_with_model_ltr_random_forest_cosine_tfidf_...  0.203121  0.206317   
rank_with_model_ltr_random_forest_cosine_tfidf_...  0.184689  0.178371   
rank_with_model_ltr_random_forest_rouge-1_r_464...  0.202013  0.205610   
rank_with_model_ltr_random_forest_rouge-1_r_464...  0.184255  0.178717   
rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb     0.215299  0.244427   
rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb...  0.215054  0.228113   
rank_with_model_ltr_svm_rouge-1_r_8dd264b4d2        0.217884  0.229743   
rank_with_model_ltr_svm_rouge-1_r_8dd264b4d2_pr...  0.209460  0.205204   
upper_bound                                         0.105305  0.067818   

metric                                                         rouge-2  \
metric_type                                                r         f   
experiment                                                               
baseline_rank_cosine_tfidf                          0.194852  0.043968   
baseline_rank_cosine_tfidf_processed                0.217775  0.062235   
baseline_rank_rouge-1_r                             0.225864  0.046198   
baseline_rank_rouge-1_r_processed                   0.243826  0.060661   
key_events_summaries_1                              0.120764  0.037226   
key_events_summaries_1_processed                    0.152725  0.062947   
key_events_summaries_2                              0.120376  0.037130   
key_events_summaries_2_processed                    0.151988  0.062645   
key_events_summaries_3                              0.120764  0.037226   
key

In [14]:
evaluation.bound_metrics()

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/upper_bound.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/upper_bound_avg.pickle


In [26]:
pd_bound = evaluation.output_avg_bound()

In [27]:
pd_bound

,metric,metric_type,value
0,rouge-1,f,0.105305
1,rouge-1,p,0.067818
2,rouge-1,r,0.361353
3,rouge-2,f,0.021309
4,rouge-2,p,0.013826
5,rouge-2,r,0.073771
6,rouge-l,f,0.218348
7,rouge-l,p,0.228750
8,rouge-l,r,0.232812
